In [119]:
import numpy as np
import csv
import matplotlib.pyplot as plt
import random
import tensorflow as tf
from bayes_opt import BayesianOptimization
from skopt import gp_minimize
from skopt.space import Real, Integer
from functools import partial

In [7]:
overall_dict = {}

overall_dict["Fat"] = 0
overall_dict["Saturated fatty acids"] = 0
overall_dict["Fatty acids, total trans"] = 0
overall_dict["Cholesterol"] = 0
overall_dict["Sodium"] = 0
overall_dict["Carbohydrate"] = 0
overall_dict["Fiber"] = 0
overall_dict["Sugars"] = 0
overall_dict["Protein"] = 0
overall_dict["Calcium"] = 0
overall_dict["Iron"] = 0
overall_dict["Potassium"] = 0
overall_dict["Vitamin D"] = 0
overall_dict["Weight"] = 0

In [8]:
tates_dict = overall_dict.copy()
tates_dict["Fat"] = 7
tates_dict["Saturated fatty acids"] = 4.5
tates_dict["Fatty acids, total trans"] = 0
tates_dict["Cholesterol"] = .025
tates_dict["Sodium"] = .16
tates_dict["Carbohydrate"] = 18
tates_dict["Fiber"] = .8
tates_dict["Sugars"] = 12
tates_dict["Protein"] = 2
tates_dict["Calcium"] = .0000001
tates_dict["Iron"] = .0009
tates_dict["Potassium"] = .01
tates_dict["Vitamin D"] = .05
tates_dict["Weight"] = 28

In [9]:
ingredient_dict = overall_dict.copy()

def reader(file,ingredient_dict,desired_serving):
    with open(file, 'r') as csvfile:
        csvreader = csv.reader(csvfile)
        for i,row in enumerate(csvreader):
            if len(row) > 0 and row[0] in ingredient_dict:
                if row[2] == "mg":
                    ingredient_dict[row[0]] = (float(row[1]) / 1000)
                elif row[2] == "mcg":
                    ingredient_dict[row[0]] = (float(row[1]) / 1000000)
                else: #grams
                    ingredient_dict[row[0]] = (float(row[1]))
            if i == 4:
                index = row.index("g")
                ingredient_dict["Weight"] = float(row[index-1])
    return ingredient_dict

In [10]:
semi_sweet_chocolate_file = "semisweet_chocolate_chips_by_raleys.csv"
semi_sweet_chocolate_dict = overall_dict.copy()
reader(semi_sweet_chocolate_file,semi_sweet_chocolate_dict,desired_serving)

unbleached_flour_file = "flour_unbleached_enriched_allpurpose_wheat.csv"
unbleached_flour_dict = overall_dict.copy()
reader(unbleached_flour_file,unbleached_flour_dict,desired_serving)

salted_butter_file = "butter_salted.csv"
salted_butter_dict = overall_dict.copy()
reader(salted_butter_file,salted_butter_dict,desired_serving)

cane_sugar_file = "granulated_pure_cane_sugar.csv"
cane_sugar_dict = overall_dict.copy()
reader(cane_sugar_file,cane_sugar_dict,desired_serving)

brown_cane_sugar_file = "brown_sugar_cane_by_frusecha.csv"
brown_cane_sugar_dict = overall_dict.copy()
reader(brown_cane_sugar_file,brown_cane_sugar_dict,desired_serving)

eggs_file = "egg_fresh_raw_whole.csv"
eggs_dict = overall_dict.copy()
reader(eggs_file,eggs_dict,desired_serving)

baking_soda_file = "leavening_agents_baking_soda.csv"
baking_soda_dict = overall_dict.copy()
reader(baking_soda_file,baking_soda_dict,desired_serving)

salt_file = "salt_table.csv"
salt_dict = overall_dict.copy()
reader(salt_file,salt_dict,desired_serving)

natural_vanilla_flavor_file = "vanilla_flavoring_syrup_by_r_torre__coinc.csv"
natural_vanilla_flavor_dict = overall_dict.copy()
reader(natural_vanilla_flavor_file,natural_vanilla_flavor_dict,desired_serving)

vanilla_extract_file = "vanilla_extract.csv"
vanilla_extract_dict = overall_dict.copy()
reader(vanilla_extract_file,vanilla_extract_dict,desired_serving)

dictionary_list = [semi_sweet_chocolate_dict,unbleached_flour_dict,salted_butter_dict,cane_sugar_dict,brown_cane_sugar_dict,eggs_dict,baking_soda_dict,salt_dict,vanilla_extract_dict]
heavy_dictionary_list = [semi_sweet_chocolate_dict,unbleached_flour_dict,salted_butter_dict,cane_sugar_dict,brown_cane_sugar_dict,eggs_dict]
key_list = ["Fat","Saturated fatty acids","Fatty acids, total trans","Cholesterol","Sodium","Carbohydrate","Fiber","Sugars","Protein","Calcium","Iron","Potassium","Vitamin D","Weight"]
key_list_sodium = ["Fat","Saturated fatty acids","Fatty acids, total trans","Cholesterol","Carbohydrate","Fiber","Sugars","Protein","Calcium","Iron","Potassium","Vitamin D","Weight"]

In [11]:
def building_x_list(number,dictionary_list):
    x_list = []
    for variable in range(0, len(dictionary_list)):
        x_list.append(number)
    return x_list

x_list = building_x_list(1,dictionary_list)

In [38]:
def equations(dictionary_list,x_list,desired_dict,key_list):
    list_of_equations = []
    for key in key_list:
        equation = 0
        for i,dictionary in enumerate(dictionary_list):
            equation = equation + (dictionary[key] * (x_list[i] / 10))
        list_of_equations.append(equation-(desired_dict[key]))
    return list_of_equations

In [33]:
def initialize_variables_tf(n,desired_dict):
    largest_weight = desired_dict["Weight"]

    initial_values = tf.random.uniform(shape=(n,), minval=0.0, maxval=largest_weight)
    initial_values = tf.sort(initial_values, direction='DESCENDING')

    initial_values = (initial_values * largest_weight) / tf.reduce_sum(initial_values)
    
    return tf.Variable(initial_values, trainable=True)

In [ ]:
def loss_tf(variables_tf,predicted):

    error = tf.reduce_mean(tf.square(predicted))
    error = error +  5*(tf.reduce_sum(tf.square(tf.minimum(variables_tf, 0))))
    
    variables_tf_array = variables_tf.numpy()
    for i in range(0,len(variables_tf_array)-2):
        if variables_tf_array[i] - variables_tf_array[i + 1] < 0:
            error = error + ((variables_tf_array[i] - variables_tf_array[i + 1]) ** 2)

    return error

In [147]:
def running_tf(dictionary_list,target_dict,key_list):  
    #number of ingredients
    n = len(dictionary_list)
    #first guess
    variables_tf = initialize_variables_tf(n,target_dict)
    print(variables_tf)
    print(tf.reduce_sum(variables_tf).numpy())
    #the optimizer chosen: stochastic gradient descent. WHAT IF WE TRIED batch gradient descent here?
    #pros/cons: https://www.geeksforgeeks.org/difference-between-batch-gradient-descent-and-stochastic-gradient-descent/
    #whats the math behind this?
        #updates each variable. the variable is updated to be its value - 
        #the learning rate * the gradient of how much that variable affects the loss.
        #THIS PART IS DIFFERENT FROM MY PREVIOUS METHOD MATHEMATICALLY, but does the same thing
    optimizer = tf.keras.optimizers.SGD(learning_rate=.01)

# Training loop
    #redefines the variable 400 times
    for epoch in range(600):

        #the "with" allows for automatic differentiation. this means the with part locks in the operations. but here no gradients are calculated.
        with tf.GradientTape() as tape:
        
            predictions1 = equations(dictionary_list,variables_tf,target_dict,key_list)
            predictions = tf.convert_to_tensor(predictions1)
            loss = loss_tf(variables_tf,predictions)

        #gradients are computed here.
        gradients = tape.gradient(loss, [variables_tf])

        #apply these gradients,perform sgd on our variables (which are being updated) 
        optimizer.apply_gradients(zip(gradients, [variables_tf]))

        #if epoch % 20 == 0:
         #   print((tf.reduce_sum(variables_tf).numpy()))
        #    print(f"Epoch {epoch}, Loss: {loss.numpy():.4f}, Variables: {variables_tf.numpy()}")

    return variables_tf.numpy().tolist()

In [79]:
running_tf(heavy_dictionary_list,tates_dict,key_list)

<tf.Variable 'Variable:0' shape=(6,) dtype=float32, numpy=
array([6.8574424 , 6.3201056 , 5.168346  , 5.0650344 , 4.2372456 ,
       0.35182598], dtype=float32)>
28.0
28.002226
Epoch 0, Loss: 0.1289, Variables: [6.8579583  6.32034    5.1698413  5.064878   4.237122   0.35208455]
28.042135
Epoch 20, Loss: 0.1240, Variables: [6.8674717  6.3243556  5.1986675  5.0610747  4.233984   0.35658067]
28.07469
Epoch 40, Loss: 0.1197, Variables: [6.8756933  6.327295   5.2256327  5.0562763  4.2298436  0.35995087]
28.101528
Epoch 60, Loss: 0.1159, Variables: [6.8829193  6.3294086  5.250982   5.0508175  4.2250104  0.36239007]
28.123894
Epoch 80, Loss: 0.1124, Variables: [6.889376   6.330891   5.2749124  5.0449476  4.219717   0.36405084]
28.142754
Epoch 100, Loss: 0.1093, Variables: [6.895236  6.331889  5.2975826 5.0388517 4.2141395 0.3650527]
28.158833
Epoch 120, Loss: 0.1065, Variables: [6.900631   6.3325152  5.319119   5.0326695  4.2084074  0.36549032]
28.172693
Epoch 140, Loss: 0.1039, Variables: [6

[6.990626811981201,
 6.32260274887085,
 5.638472557067871,
 4.934759140014648,
 4.102799415588379,
 0.2985755205154419]

In [80]:
def add_noise(optimized_heavy_ingredients):
    for i in range(0,len(optimized_heavy_ingredients)):
        number = random.randint(1,2)
        if number == 1:
            optimized_heavy_ingredients[i] -= .01
        else:
            optimized_heavy_ingredients[i] += .01
    return optimized_heavy_ingredients

In [128]:
opti = [6.990626811981201,
 6.32260274887085,
 5.638472557067871,
 4.934759140014648,
 4.102799415588379,
 0.2985755205154419]

In [143]:
def error_function_lighter(lighter_ingredients,optimized_heavy_ingredients):

    full_ingredients = np.concatenate((optimized_heavy_ingredients, lighter_ingredients))
    equations_list = equations(dictionary_list,full_ingredients,tates_dict,key_list)
    error = 0
    for i, equation in enumerate(equations_list):
        #if i == 4 or i == 5:
        #    error += equation ** 2
        #else:
        error += equation

    #gives the total carbs/fats/proteins as looks at the difference + targeted amount --> actual amount
    carbs = equations_list[5] + tates_dict["Carbohydrate"]
    fats = equations_list[0] + tates_dict["Fat"]
    proteins = equations_list[8] + tates_dict["Protein"]
    calorie_count = (carbs * 4) + (fats * 9) + (proteins * 4)
    #the 140 is tates specific
    difference_in_calories = ((calorie_count) - 140) ** 2
    error += (difference_in_calories / 100)   
    #error /= len(equations_list)
    return abs(error)

In [145]:
def error_function_bayesian(lighter_ingredients, optimized_heavy_ingredients):
    return error_function_lighter(lighter_ingredients, optimized_heavy_ingredients)

# Map lighter ingredients to their indices
lighter_indices = [6, 7, 8]  # vanilla, baking soda, xanthan gum

# Define the search space for Bayesian Optimization using the indices
search_space = [
    Real(0.00001, 1, name=f"ingredient_{lighter_indices[0]}"),  # baking_soda
    Real(0.00001, 1, name=f"ingredient_{lighter_indices[1]}"),  # salt
    Real(0.00001, 1, name=f"ingredient_{lighter_indices[2]}")   # vanilla
]

def bayesiann(optimized_heavy_ingredients):
    print("Optimized heavy ingredients passed to bayesiann:")
    print(optimized_heavy_ingredients)

    error_function_fixed = partial(error_function_lighter, optimized_heavy_ingredients=optimized_heavy_ingredients)
    result = gp_minimize(
        func=error_function_fixed,
        dimensions=search_space,
        #number of combinations tried
        n_calls=50,
        #setting to 10 random guesses first, allowing exploration before making "smarter decisions"
        n_random_starts=10,
        #410 doesn't matter (just my bday). convention is 42.
        #random state ensures that the random first combinations are the same for
        #each run: pros
            #Ensures repeatability (same experiment = same results).
            # if results change, it's not due to randomness.
            # Allows fair comparisons (changing only one parameter won't affect randomness).
        #random_state=410
    )
    optimized_lighter_ingredients = result.x
    print("Debug: Optimized lighter ingredients from Bayesian Optimization:")
    print(result.x)
    
    # Debug: Print the minimum error achieved
    print("Debug: Minimum error achieved:")
    print(result.fun)
    return optimized_lighter_ingredients

In [156]:
eh = [7.7850305938720705, 6.252763519287109, 5.657139053344727, 5.837734928131104, 2.373724689483643, 0.3693550431728363]

In [157]:
bayesiann(eh)

Optimized heavy ingredients passed to bayesiann:
[7.7850305938720705, 6.252763519287109, 5.657139053344727, 5.837734928131104, 2.373724689483643, 0.3693550431728363]
Debug: Optimized lighter ingredients from Bayesian Optimization:
[0.5357411429470648, 1e-05, 0.579714860265312]
Debug: Minimum error achieved:
0.0007517155807461179


[0.5357411429470648, 1e-05, 0.579714860265312]

In [154]:
def reversy(dictionary_list_for_heavy,target_dict,key_list):

    optimized_heavy = running_tf(dictionary_list_for_heavy,target_dict,key_list)
    noisy_optimized = add_noise(optimized_heavy)
    lighter = bayesiann(noisy_optimized)

    return optimized_heavy, lighter
    

In [155]:
reversy(heavy_dictionary_list,tates_dict,key_list)

<tf.Variable 'Variable:0' shape=(6,) dtype=float32, numpy=
array([7.640944 , 6.1839886, 5.6975045, 5.6498637, 2.257335 , 0.5703667],
      dtype=float32)>
28.000002
Optimized heavy ingredients passed to bayesiann:
[7.7850305938720705, 6.252763519287109, 5.657139053344727, 5.837734928131104, 2.373724689483643, 0.3693550431728363]
Debug: Optimized lighter ingredients from Bayesian Optimization:
[1e-05, 1.0, 1e-05]
Debug: Minimum error achieved:
0.007327536285742217


([7.7850305938720705,
  6.252763519287109,
  5.657139053344727,
  5.837734928131104,
  2.373724689483643,
  0.3693550431728363],
 [1e-05, 1.0, 1e-05])

In [166]:
smoothie_dict = overall_dict.copy()
smoothie_dict["Fat"] = 10.1
smoothie_dict["Saturated fatty acids"] = 2.5
smoothie_dict["Fatty acids, total trans"] = 0
smoothie_dict["Cholesterol"] = 10/1000
smoothie_dict["Sodium"] = 428.7/1000
smoothie_dict["Carbohydrate"] = 92.2
smoothie_dict["Fiber"] = 10.2
smoothie_dict["Sugars"] = 66.9 + 37.5
smoothie_dict["Protein"] = 30.7
smoothie_dict["Calcium"] = 870/1000
smoothie_dict["Iron"] = 3/1000
smoothie_dict["Potassium"] = 1205.7/1000
smoothie_dict["Vitamin D"] = 0
smoothie_dict["Weight"] = 665.1

In [167]:
vanilla_almond_milk_file = "vanilla_almond_milk_by_topco_associates_inc.csv"
vanilla_almond_milk_dict = overall_dict.copy()
reader(vanilla_almond_milk_file,vanilla_almond_milk_dict,desired_serving)

frozen_blueberries_file = "blueberries_frozen.csv"
frozen_blueberries_dict = overall_dict.copy()
reader(frozen_blueberries_file,frozen_blueberries_dict,desired_serving)

frozen_strawberries_file = "strawberries_frozen.csv"
frozen_strawberries_dict = overall_dict.copy()
reader(frozen_strawberries_file,frozen_strawberries_dict,desired_serving)

banana_file = "banana_raw.csv"
banana_dict = overall_dict.copy()
reader(banana_file,banana_dict,desired_serving)

isopure_zero_carb_file = "isopure_zero_carb_protein_creamy_vanilla.csv"
isopure_zero_carb_dict = overall_dict.copy()
reader(isopure_zero_carb_file,isopure_zero_carb_dict,desired_serving)

granola_homemade_file = "granola_homemade.csv"
granola_homemade_dict = overall_dict.copy()
reader(granola_homemade_file,granola_homemade_dict,desired_serving)

honey_file = "honey.csv"
honey_dict = overall_dict.copy()
reader(honey_file,honey_dict,desired_serving)

smoothie_ingredient_list = [vanilla_almond_milk_dict, frozen_blueberries_dict, frozen_strawberries_dict, banana_dict, isopure_zero_carb_dict, granola_homemade_dict, honey_dict]


In [168]:
running_tf(smoothie_ingredient_list,smoothie_dict,key_list)

<tf.Variable 'Variable:0' shape=(7,) dtype=float32, numpy=
array([271.92798 , 196.73726 ,  69.600945,  65.63423 ,  35.12482 ,
        19.13118 ,   6.943507], dtype=float32)>
665.09985


[267.6542053222656,
 193.63507080078125,
 65.10347747802734,
 64.10382843017578,
 28.462602615356445,
 21.039901733398438,
 31.2152156829834]

In [169]:
running_tf(smoothie_ingredient_list,smoothie_dict,key_list)

<tf.Variable 'Variable:0' shape=(7,) dtype=float32, numpy=
array([151.26909 , 145.85268 , 141.86902 , 102.03573 ,  76.58154 ,
        24.314064,  23.17786 ], dtype=float32)>
665.1


[152.32650756835938,
 147.1532440185547,
 142.36123657226562,
 103.30818939208984,
 57.0115966796875,
 22.126388549804688,
 39.401920318603516]

In [ ]:
actual = [244,138,132,68,31,30,21]